## Finnhub Streaming Api

In [ ]:
import json
import pandas as pd
import websocket

def on_message(ws, message):
    js = json.loads(message)
    data = js.pop("data")
    df = pd.DataFrame(data)
    df = df.rename(
                columns={
                    "s": "symbol",
                    "p": "last_price",
                    "t": "timestamp",
                    "v": "volume",
                    "t": "timestamp",
                    "c": "trade_conditions",
                }
    )
    print(df)

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    ws.send('{"type":"subscribe","symbol":"GME"}')
    ws.send('{"type":"subscribe","symbol":"AAPL"}')
    ws.send('{"type":"subscribe","symbol":"AMZN"}')
    ws.send('{"type":"subscribe","symbol":"BINANCE:BTCUSDT"}')
    ws.send('{"type":"subscribe","symbol":"IC MARKETS:1"}')

if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://ws.finnhub.io?token=c1ljb2237fkt2msk6ueg",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

## Stock Candle Unit Testing

In [12]:
import os
import configparser
from datetime import datetime, timedelta

import finnhub
import pandas as pd


FINNHUB_FILE_PATH = "foobar/data/raw/"

config = configparser.ConfigParser()
config.read("../foobar/data_loader/conf/finnhub.cfg")
api_credential = config["api_credential"]
AUTH_TOKEN = api_credential["auth_token"]

api_client = finnhub.Client(api_key=AUTH_TOKEN)



date_to = datetime(year=2020, month=12, day=1)
date_from = datetime(year=2020, month=3, day=1)

from_ts = int(datetime.timestamp(date_from))
to_ts = int(datetime.timestamp(date_to))

df = pd.DataFrame(
    api_client.stock_candles(
        symbol='GME', resolution='D', _from=from_ts, to=to_ts
    )
)
# print(df)
df = df.rename(
    columns={
        "c": "close_price",
        "o": "open_price",
        "h": "high-price",
        "l": "low-price",
        "v": "volume",
        "t": "timestamp",
        "s": "status",
    }
)
# pd.to_datetime(df['timestamp'])
# df['timestamp'].fromtimestamp()
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df

,close_price,high-price,low-price,open_price,status,timestamp,volume,datetime
0,3.83,3.85,3.3900,3.60,ok,1583107200,3208428,2020-03-02
1,3.63,3.89,3.5000,3.88,ok,1583193600,4282456,2020-03-03
2,3.77,3.83,3.4600,3.71,ok,1583280000,4257629,2020-03-04
3,3.96,3.99,3.5900,3.70,ok,1583366400,3599627,2020-03-05
4,3.92,4.07,3.7600,3.84,ok,1583452800,2096803,2020-03-06
...,...,...,...,...,...,...,...,...
187,13.67,14.26,13.3000,14.23,ok,1606176000,7186710,2020-11-24
188,14.75,15.25,13.4200,13.60,ok,1606262400,8860092,2020-11-25
189,16.08,16.74,14.8800,15.29,ok,1606435200,12504197,2020-11-27
190,16.56,19.42,16.5575,18.17,ok,1606694400,31983498,2020-11-30


In [ ]:
class finnhub_dataloader:
    def __init__(self, api_token):
        self.api_client = finnhub.Client(api_key=api_token)

    def get_filling_sentiment(dataloader, date_from, date_to):
        from_str = date_from.strftime("%Y-%m-%d")
        to_str = date_to.strftime("%Y-%m-%d")
        if os.path.exists(FINNHUB_FILE_PATH + f"filling_sentiment_{from_str}_{to_str}.csv"):
            print("SEC sentiment analysis dataset is already created.")
        else:
            sec_filling_sentiment_timeseries = dataloader.query_filling_sentiment(
                symbol="GME", date_from=date_from, date_to=date_to
            )
            sec_filling_sentiment_timeseries.to_csv(
                FINNHUB_FILE_PATH + f"filling_sentiment_{from_str}_{to_str}.csv",
                encoding="utf-8",
            )
            print("SEC sentiment analysis dataset is successfully created.")


In [ ]:
dataloader = finnhub_dataloader(api_token=AUTH_TOKEN)
get_filling_sentiment(dataloader, date_from, date_to)